In [ ]:
import random
import datetime
from math import sqrt
from functools import lru_cache
from collections import deque, defaultdict

class Cell:
    def __init__(self, x, y):
        self.x = x
        self.y = y
        self.ent = None

    def empty(self):
        return self.ent is None

class Entity:
    def __init__(self, color):
        self.color = color

class Eater(Entity): pass

class Food(Entity):
    def __init__(self, color, is_last=False):
        super().__init__(color)
        self.is_last = is_last


class Grid:
    DIRS = [(0,1), (0,-1), (1,0), (-1,0)]
    COLOR_MAP = {
        'Food': {'White': 0, 'Green': 1, 'Blue': 2, 'Red': 3, 'Yellow': 4, 'Purple': 5, 'Pink': 6, 'Brown': 7},
        'Eater': {'Green': 0, 'Blue': 1, 'Red': 2, 'Yellow': 3, 'Purple': 4, 'Pink': 5, 'Brown': 6}
    }
    
    # Default scoring weights
    DEFAULT_WEIGHTS = {
        'food_separation': 5.0,
        'cluster_size': 0.33,
        'clusters_count': 1.0,
        'distribution': 1.0,
        'distance_to_eater': 1.0,
        'path_complexity': 0.8,
        'interaction_complexity': 1.5,
        'empty_cells': 2.0  # Weight for empty cells score
    }

    def __init__(self, rows, cols, weights=None):
        self.rows = rows
        self.cols = cols
        self.grid = [[Cell(x, y) for y in range(cols)] for x in range(rows)]
        self.moves = []
        self.min_separation = 2
        self._distance_cache = {}
        self._cluster_cache = {}
        self._path_cache = {}
        self._food_by_color = defaultdict(list)
        self._eater_positions = {}
        # Initialize weights with defaults or custom values
        self.weights = weights if weights is not None else self.DEFAULT_WEIGHTS.copy()
        
    def calculate_clustering_score(self, minimum_qualified_colors):
        colors = set(self.COLOR_MAP['Food'].keys()) - {'White'}
        metrics = {}
        
        # Update food locations cache
        self._update_entity_cache()
        
        for color in colors:
            foods = self._food_by_color.get(color, [])
            
            if not foods:
                continue

            eater_pos = self._eater_positions.get(color)
            
            if not eater_pos:
                continue

            # Use the cached position tuples
            foods_tuple = tuple(sorted(foods))
            
            color_metrics = {
                'food_count': len(foods),
                'avg_food_separation': self._calculate_avg_separation(foods_tuple),
                'largest_cluster_size': self._find_largest_cluster(foods_tuple),
                'clusters_count': self._count_clusters(foods_tuple),
                'avg_distance_to_eater': self._calculate_avg_distance_to_point(foods, eater_pos),
                'distribution_score': self._calculate_distribution_score(foods),
                'path_complexity': self._calculate_path_complexity(color)
            }
            
            if (color_metrics['largest_cluster_size'] == color_metrics['food_count'] and 
                color_metrics['food_count'] > 1):
                color_metrics['disqualified'] = True
            else:
                color_metrics['disqualified'] = False
                
            metrics[color] = color_metrics

        if not metrics:
            return {'final_score': float('inf'), 'color_metrics': {}, 'disqualified': True}

        disqualified = sum(0 if m['disqualified'] else 1 for m in metrics.values()) < minimum_qualified_colors
        
        # Calculate interaction complexity between different color eaters
        interaction_complexity = self._calculate_eater_interactions()
        
        # Calculate empty cells score
        empty_cells = self.count_empty()
        empty_cells_score = self._calculate_empty_cells_score(empty_cells)
        
        final_score = float('inf') if disqualified else self._calculate_final_score(
            metrics, interaction_complexity, empty_cells_score)
        
        return {
            'final_score': final_score,
            'color_metrics': metrics,
            'disqualified': disqualified,
            'interaction_complexity': interaction_complexity,
            'empty_cells': empty_cells,
            'empty_cells_score': empty_cells_score
        }
    
    def _calculate_empty_cells_score(self, empty_cells):
        """Calculate a score based on how few empty cells are in the board.
        Fewer empty cells = higher score."""
        total_cells = self.rows * self.cols
        filled_ratio = 1 - (empty_cells / total_cells)
        # Higher score for more filled boards (fewer empty cells)
        return filled_ratio * 100
    
    def _update_entity_cache(self):
        """Update cached positions of foods by color and eaters"""
        self._food_by_color.clear()
        self._eater_positions.clear()
        
        for x in range(self.rows):
            for y in range(self.cols):
                cell = self.grid[x][y]
                if isinstance(cell.ent, Food):
                    self._food_by_color[cell.ent.color].append((x, y))
                elif isinstance(cell.ent, Eater):
                    self._eater_positions[cell.ent.color] = (x, y)

    @lru_cache(maxsize=4096)
    def _manhattan_distance(self, pos1, pos2):
        return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

    def _calculate_avg_separation(self, positions):
        if len(positions) < 2:
            return float('inf')
        
        cache_key = positions
        if cache_key in self._distance_cache:
            return self._distance_cache[cache_key]
            
        distances = []
        for i, pos1 in enumerate(positions[:-1]):
            for pos2 in positions[i+1:]:
                distances.append(self._manhattan_distance(pos1, pos2))
        
        result = sum(distances) / len(distances) if distances else float('inf')
        self._distance_cache[cache_key] = result
        return result

    def _count_clusters(self, positions, cluster_threshold=2):
        if not positions:
            return 0
            
        cache_key = (positions, cluster_threshold)
        if cache_key in self._cluster_cache:
            return self._cluster_cache[cache_key]
            
        positions_set = set(positions)
        clusters = 0
        
        while positions_set:
            clusters += 1
            start = next(iter(positions_set))
            positions_set.remove(start)
            
            queue = deque([start])
            while queue:
                current = queue.popleft()
                for pos in list(positions_set):
                    if self._manhattan_distance(current, pos) <= cluster_threshold:
                        queue.append(pos)
                        positions_set.remove(pos)
        
        self._cluster_cache[cache_key] = clusters
        return clusters

    def _find_largest_cluster(self, positions, cluster_threshold=2):
        if not positions:
            return 0
            
        cache_key = (positions, cluster_threshold)
        if cache_key in self._cluster_cache:
            return self._cluster_cache.get(f"largest_{cache_key}", 0)
            
        positions_set = set(positions)
        largest_cluster = 0
        
        while positions_set:
            start = next(iter(positions_set))
            positions_set.remove(start)
            
            queue = deque([start])
            cluster_size = 1
            
            while queue:
                current = queue.popleft()
                for pos in list(positions_set):
                    if self._manhattan_distance(current, pos) <= cluster_threshold:
                        queue.append(pos)
                        positions_set.remove(pos)
                        cluster_size += 1
            
            largest_cluster = max(largest_cluster, cluster_size)
        
        self._cluster_cache[f"largest_{cache_key}"] = largest_cluster
        return largest_cluster

    def _calculate_avg_distance_to_point(self, positions, target):
        if not positions:
            return 0
        distances = [self._manhattan_distance(pos, target) for pos in positions]
        return sum(distances) / len(distances)

    def _calculate_distribution_score(self, positions):
        if not positions:
            return 0
            
        mid_row = self.rows // 2
        mid_col = self.cols // 2
        quadrants = [0] * 4
        
        for x, y in positions:
            quad_idx = (x >= mid_row) * 2 + (y >= mid_col)
            quadrants[quad_idx] += 1
            
        avg = sum(quadrants) / 4
        variance = sum((q - avg) ** 2 for q in quadrants) / 4
        return sqrt(variance)
    
    def _calculate_path_complexity(self, color):
        """Calculate the complexity of the path an eater must take to collect its food"""
        eater_pos = self._eater_positions.get(color)
        if not eater_pos:
            return 0
            
        foods = self._food_by_color.get(color, [])
        if not foods:
            return 0
            
        # Cache key for this specific configuration
        cache_key = (eater_pos, tuple(sorted(foods)))
        if cache_key in self._path_cache:
            return self._path_cache[cache_key]
            
        # Find optimal path
        current_pos = eater_pos
        remaining_foods = set(foods)
        total_distance = 0
        direction_changes = 0
        prev_direction = None
        
        while remaining_foods:
            # Find nearest food
            nearest_food = min(remaining_foods, key=lambda pos: self._manhattan_distance(current_pos, pos))
            remaining_foods.remove(nearest_food)
            
            # Calculate path metrics
            dx = nearest_food[0] - current_pos[0]
            dy = nearest_food[1] - current_pos[1]
            
            # Calculate direction changes
            if dx != 0 and dy != 0:
                # Both directions need change
                direction_changes += 1
                
            current_direction = None
            if abs(dx) > abs(dy):
                current_direction = "horizontal"
            elif abs(dy) > abs(dx):
                current_direction = "vertical"
                
            if prev_direction and current_direction and prev_direction != current_direction:
                direction_changes += 1
                
            prev_direction = current_direction
            
            # Update position and add distance
            total_distance += abs(dx) + abs(dy)
            current_pos = nearest_food
            
        complexity = total_distance * 0.5 + direction_changes * 2.0
        self._path_cache[cache_key] = complexity
        return complexity
    
    def _calculate_eater_interactions(self):
        """Calculate the complexity of interactions between eaters"""
        eaters = list(self._eater_positions.items())
        if len(eaters) < 2:
            return 0
            
        interaction_score = 0
        
        # Calculate space contention between eaters
        for i, (color1, pos1) in enumerate(eaters[:-1]):
            for color2, pos2 in eaters[i+1:]:
                # Eaters that are close might contend for space
                distance = self._manhattan_distance(pos1, pos2)
                if distance < self.rows + self.cols / 3:
                    interaction_score += (self.rows + self.cols) / (distance + 1)
                    
                # Check for potential path crossing
                foods1 = set(self._food_by_color.get(color1, []))
                foods2 = set(self._food_by_color.get(color2, []))
                
                # Check if eaters need to cross paths to reach their foods
                for f1 in foods1:
                    for f2 in foods2:
                        # If paths might intersect
                        if (self._manhattan_distance(pos1, f2) + self._manhattan_distance(pos2, f1) <
                            self._manhattan_distance(pos1, f1) + self._manhattan_distance(pos2, f2)):
                            interaction_score += 5
        
        # Bonus for eaters that are surrounded by other colors' food
        for color, pos in eaters:
            other_foods = []
            for c, foods in self._food_by_color.items():
                if c != color and c != 'White':
                    other_foods.extend(foods)
            
            # Count nearby foods of other colors
            nearby_other_foods = sum(1 for f in other_foods if self._manhattan_distance(pos, f) < 3)
            interaction_score += nearby_other_foods * 2
                    
        return interaction_score

    def _calculate_final_score(self, metrics, interaction_complexity, empty_cells_score):
        scores = []
        for color_metrics in metrics.values():
            if color_metrics.get('disqualified', False):
                continue
                a
            color_score = (
                (self.weights['food_separation'] / (color_metrics['avg_food_separation'] + 0.1)) +
                (color_metrics['largest_cluster_size'] * self.weights['cluster_size']) +
                (self.weights['clusters_count'] / (color_metrics['clusters_count'] + 0.1)) +
                (color_metrics['distribution_score'] * self.weights['distribution']) +
                (self.weights['distance_to_eater'] / (color_metrics['avg_distance_to_eater'] + 1)) +
                (color_metrics['path_complexity'] * self.weights['path_complexity'])
            )
            scores.append(color_score)
        
        base_score = sum(scores) / len(scores) if scores else float('inf')
        
        # Add interaction complexity and empty cells score to the final score
        return (base_score + 
               (interaction_complexity * self.weights['interaction_complexity']) +
               (empty_cells_score * self.weights['empty_cells']))
    
    # Method to update weights
    def set_weights(self, new_weights):
        """Update the scoring weights with new values"""
        for key, value in new_weights.items():
            if key in self.weights:
                self.weights[key] = value
    
    def place(self, ent, x, y):
        self.grid[x][y].ent = ent
        # Update entity cache after placement
        if isinstance(ent, Food):
            self._food_by_color[ent.color].append((x, y))
        elif isinstance(ent, Eater):
            self._eater_positions[ent.color] = (x, y)

    def clear(self):
        for row in self.grid:
            for cell in row: cell.ent = None
        # Clear caches
        self._distance_cache.clear()
        self._cluster_cache.clear()
        self._path_cache.clear()
        self._food_by_color.clear()
        self._eater_positions.clear()

    def gen_puzzle(self, eaters, white_pct, min_food):
        self.clear()
        eater_pos = []
        
        colors = [c for c, n in eaters.items() for _ in range(n)]
        random.shuffle(colors)
        
        # Place eaters first
        for color in colors:
            pos = self.get_empty()
            if not pos: break
            x, y = pos
            e = Eater(color)
            self.place(e, x, y)
            eater_pos.append({'e': e, 'x': x, 'y': y, 'count': 0})

        # First pass: ensure minimum food count
        active_eaters = eater_pos.copy()
        while active_eaters:
            moved_any = False
            # Shuffle eaters each round to create more interesting interactions
            random.shuffle(active_eaters)
            
            for einfo in active_eaters[:]:
                if einfo['count'] >= min_food:
                    active_eaters.remove(einfo)
                    continue
                    
                # Try to move using strategic placement
                if self.move_eater_strategically(einfo, eater_pos):
                    moved_any = True
                else:
                    active_eaters.remove(einfo)
            
            if not moved_any:
                break

        # Second pass: fill remaining spaces where possible
        remaining_spaces = self.count_empty()
        active_eaters = [e for e in eater_pos if self.can_move_eater(e)]
        random.shuffle(active_eaters)
        
        failures = 0
        max_failures = 10
        
        while remaining_spaces > 0 and active_eaters and failures < max_failures:
            moved_any = False
            
            for einfo in active_eaters[:]:
                if self.move_eater_strategically(einfo, eater_pos):
                    moved_any = True
                    remaining_spaces -= 1
                    failures = 0
                else:
                    active_eaters.remove(einfo)
                    
                if remaining_spaces <= 0:
                    break
            
            if not moved_any:
                failures += 1
            
            active_eaters = [e for e in eater_pos if self.can_move_eater(e)]
            random.shuffle(active_eaters)

        self.whiten_food(white_pct)

    def can_move_eater(self, einfo):
        for dx, dy in self.DIRS:
            x, y = einfo['x'], einfo['y']
            new_x, new_y = self.get_max_pos(x, y, dx, dy)
            if new_x is not None and (new_x != x or new_y != y):
                return True
        return False

    def move_eater_strategically(self, einfo, all_eaters):
        """Move eater with improved strategic decision making for puzzle complexity"""
        random.shuffle(self.DIRS)
        best_move = None
        best_score = float('-inf')
        
        # Consider other eaters' positions for interaction complexity
        other_eaters = [(e['x'], e['y']) for e in all_eaters if e['e'].color != einfo['e'].color]
        
        for dx, dy in self.DIRS:
            x, y = einfo['x'], einfo['y']
            new_x, new_y = self.get_max_pos(x, y, dx, dy)
            
            if new_x is None or (new_x == x and new_y == y):
                continue
            
            old_ent = self.grid[x][y].ent
            self.grid[x][y].ent = Food(einfo['e'].color)
            
            # Evaluate food placement with consideration of other eaters
            move_score = self._evaluate_food_placement(x, y, einfo['e'].color)
            
            # Add bonus for moves that create interesting eater interactions
            for ex, ey in other_eaters:
                # Path blocking score - higher if this food might block another eater's path
                distance_to_other = self._manhattan_distance((x, y), (ex, ey))
                if distance_to_other < 3:
                    move_score += 20
                elif distance_to_other < 5:
                    move_score += 10
            
            self.grid[x][y].ent = old_ent
            
            if move_score > best_score:
                best_score = move_score
                best_move = (new_x, new_y)
        
        # Fallback to any valid move if no good strategic move found
        if best_move is None:
            for dx, dy in self.DIRS:
                x, y = einfo['x'], einfo['y']
                new_x, new_y = self.get_max_pos(x, y, dx, dy)
                if new_x is not None and (new_x != x or new_y != y):
                    best_move = (new_x, new_y)
                    break
        
        if best_move:
            new_x, new_y = best_move
            x, y = einfo['x'], einfo['y']
            
            is_first_food = einfo['count'] == 0
            
            self.grid[x][y].ent = None
            self.place(einfo['e'], new_x, new_y)
            self.place(Food(einfo['e'].color, is_last=is_first_food), x, y)
            einfo['count'] += abs(new_x - x) + abs(new_y - y)
            
            self.moves.append({
                'eater_color': einfo['e'].color,
                'start': (new_x, new_y),
                'end': (x, y),
                'food_color': einfo['e'].color,
                'dist': abs(new_x - x) + abs(new_y - y)
            })
            
            einfo['x'], einfo['y'] = new_x, new_y
            
            # Clear path cache as it's now invalid
            self._path_cache.clear()
            return True
            
        return False
    
    def _evaluate_food_placement(self, x, y, color):
        """Evaluate how good a food placement is with improved metrics"""
        # Use the cached food positions
        same_color_foods = self._food_by_color.get(color, [])
        
        if not same_color_foods:
            return 100
        
        min_distance = min(self._manhattan_distance((x, y), (fx, fy)) for fx, fy in same_color_foods)
        
        temp_foods = same_color_foods + [(x, y)]
        temp_foods_tuple = tuple(sorted(temp_foods))
        
        cluster_size = self._find_largest_cluster(temp_foods_tuple)
        clusters_count = self._count_clusters(temp_foods_tuple)
        
        if cluster_size == len(temp_foods) and len(temp_foods) > 1:
            return -1000
        
        # Separation score
        separation_score = 0
        if min_distance >= self.min_separation:
            separation_score = 50 + min_distance * 10
        else:
            separation_score = 20 - (self.min_separation - min_distance) * 15
        
        # Prefer multiple clusters for complexity
        cluster_score = clusters_count * 20
        
        # Prefer food placement that creates more complex paths
        path_score = 0
        eater_pos = self._eater_positions.get(color)
        if eater_pos:
            # Prefer food that requires eater to change direction to reach
            dx1 = abs(eater_pos[0] - x)
            dy1 = abs(eater_pos[1] - y)
            if dx1 > 0 and dy1 > 0:
                path_score += 30  # Diagonal relationships require more moves
            
            # Prefer food that's moderately distant (not too close, not too far)
            dist = self._manhattan_distance(eater_pos, (x, y))
            if 3 <= dist <= 6:
                path_score += 25
            elif dist > 6:
                path_score += 15
        
        return separation_score + cluster_score + path_score

    def get_max_pos(self, x, y, dx, dy):
        nx, ny = x, y
        while self.in_bounds(nx + dx, ny + dy) and self.grid[nx + dx][ny + dy].empty():
            nx += dx
            ny += dy
        return (nx, ny) if (nx, ny) != (x, y) else (None, None)

    def whiten_food(self, pct):
        food = [(c.x, c.y) for row in self.grid for c in row 
                if isinstance(c.ent, Food) and c.ent.color != 'White' and not c.ent.is_last]
        to_white = random.sample(food, int(len(food) * pct / 100)) if food else []
        for x, y in to_white:
            self.grid[x][y].ent.color = 'White'

    def get_empty(self):
        empty = [(x,y) for x in range(self.rows) for y in range(self.cols) 
                if self.grid[x][y].empty()]
        return random.choice(empty) if empty else None

    def in_bounds(self, x, y):
        return 0 <= x < self.rows and 0 <= y < self.cols

    def count_empty(self):
        return sum(1 for row in self.grid for cell in row if cell.empty())

    def to_scene(self, level_id, fill_empty):
        width, height = 1440, 2560
        step = max(180, width // self.cols - 24)
        
        start_x = width/2 - step*self.cols/2 + step*0.5
        start_y = height/2 - step*self.rows/2 + step*0.5 - 40

        scene = f"""[gd_scene load_steps=4 format=3 uid="{level_id}"]

[ext_resource type="Script" path="res://Levels/Level.cs" id="lvlid"]
[ext_resource type="PackedScene" uid="uid://byatslmwbvorg" path="res://Entities/Food/Food.tscn" id="foodid"]
[ext_resource type="PackedScene" uid="uid://dd570jgysudow" path="res://Entities/Eater/Eater.tscn" id="eaterid"]

[node name="Level" type="Node"]
script = ExtResource("lvlid")
{self.solution_metadata()}

[node name="Food" type="Node" parent="."]

[node name="Eaters" type="Node" parent="."]
"""
        for x in range(self.rows):
            for y in range(self.cols):
                cell = self.grid[x][y]
                pos_x = start_x + y*step
                pos_y = start_y + x*step
                
                if isinstance(cell.ent, Food):
                    scene += f"""
[node name="Food{x}_{y}" parent="Food" instance=ExtResource("foodid")]
position = Vector2({pos_x}, {pos_y})
BoardStatePositionId = Vector2i({x}, {y})
FoodType = {self.COLOR_MAP['Food'][cell.ent.color]}
IsLast = {"true" if cell.ent.is_last else "false"}
"""
                elif isinstance(cell.ent, Eater):
                    scene += f"""
[node name="Eater{x}_{y}" parent="Eaters" instance=ExtResource("eaterid")]
position = Vector2({pos_x}, {pos_y})
EaterType = {self.COLOR_MAP['Eater'][cell.ent.color]}
BoardStatePositionId = Vector2i({x}, {y})
ValidFoodTypes = [0, {self.COLOR_MAP['Food'][cell.ent.color]}]
"""
                elif cell.ent is None and fill_empty:
                    scene += f"""
[node name="Food{x}_{y}" parent="Food" instance=ExtResource("foodid")]
position = Vector2({pos_x}, {pos_y})
BoardStatePositionId = Vector2i({x}, {y})
FoodType = 0
IsLast = false
"""
        return scene

    def solution_metadata(self):
        metadata = "metadata/solution = PackedVector2Array("
        for move in self.moves:
                metadata += f"{move["start"][0]}, {move["start"][1]}, {move["end"][0]}, {move["end"][1]}, "
        metadata = metadata[:-2] + ")"
        return metadata

def gen_boards(num_select, total_gen, rows, cols, eaters, white_pct, min_food, weights=None):
    boards = []
    attempts = 0
    max_tries = total_gen * 100
    minimum_qualified_colors = 0 if len(eaters) == 1 else len(eaters)
    max_empty_cells = 1 if len(eaters) == 1 else (int)(rows * cols * 0.1)
    
    while len(boards) < total_gen and attempts < max_tries:
        g = Grid(rows, cols, weights)
        g.gen_puzzle(eaters, white_pct, min_food)
        
        empty_count = g.count_empty()
        
        if all(
            sum(1 for row in g.grid for cell in row 
                if isinstance(cell.ent, Food) and cell.ent.color == color) >= min_food 
            for color in eaters):
            
            clustering_info = g.calculate_clustering_score(minimum_qualified_colors)
            
            # Skip disqualified boards
            if clustering_info['disqualified'] or empty_count >= max_empty_cells:
                attempts += 1
                continue

            # Combine all factors for total score
            total_score = (clustering_info['final_score'] + 
                          clustering_info.get('interaction_complexity', 0) * g.weights['interaction_complexity'] +
                          clustering_info.get('empty_cells_score', 0) * g.weights['empty_cells'])
            board_info = {
                'board': g,
                'moves': g.moves.copy(),
                'empty': empty_count,
                'clustering_score': clustering_info['final_score'],
                'metrics': clustering_info['color_metrics'],
                'interaction_complexity': clustering_info.get('interaction_complexity', 0),
                'empty_cells_score': clustering_info.get('empty_cells_score', 0),
                'total_score': total_score
            }
            
            
            boards.append(board_info)
        attempts += 1
    
    boards.sort(key=lambda x: x['total_score'])
    return boards[:num_select]


# Example usage
if __name__ == "__main__":
    DIFFICULTY = {
        'tutorial': ({'Green': 1}, 3, 3, 5, 30, 1),
        'very_easy': ({'Blue': 1, 'Green': 1}, 4, 4, 15, 30, 2),
        'very_easy2': ({'Blue': 1, 'Green': 1}, 5, 4, 15, 30, 2),
        'easy': ({'Blue': 1, 'Green': 1, 'Red': 1}, 5, 5, 15, 30, 3),
        'easy2': ({'Blue': 1, 'Green': 1, 'Red': 1}, 6, 5, 15, 30, 3),
        'medium': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}, 6, 6, 30, 20, 3),
        'medium2': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1}, 7, 6, 30, 20, 3),
        'hard': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1}, 7, 7, 30, 20, 3),
        'hard2': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1}, 8, 7, 30, 20, 4),
        'veryhard': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 9, 7, 45, 20, 4),
        'veryhard2': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 10, 7, 45, 20, 4),
        'max': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 11, 7, 45, 15, 4),
        'max2': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1}, 11, 7, 45, 10, 5),
        'max3': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1, 'Brown': 1}, 11, 7, 45, 15, 4),
        'max4': ({'Blue': 1, 'Green': 1, 'Red': 1, 'Yellow': 1, 'Pink': 1, 'Purple': 1, 'Brown': 1}, 11, 7, 45, 10, 5)
    }


    CUSTOM_WEIGHTS = {
        'food_separation': 3.0,
        'cluster_size': 0.5,
        'clusters_count': 1.0,
        'distribution': 1.0,
        'distance_to_eater': 1.2,
        'path_complexity': 2,
        'interaction_complexity': 2,
        'empty_cells': 2
    }

    lvlid = 1
    ts = datetime.datetime.now()
    print(f"[{ts}] Starting")
    for diff, (eaters, rows, cols, num_select, white_pct, min_food) in DIFFICULTY.items():
        start_time = datetime.datetime.now()
        new_boards = []
        while (len(new_boards) < num_select):
            new_boards += gen_boards(num_select-len(new_boards), 5000, rows, cols, eaters, white_pct, min_food, CUSTOM_WEIGHTS)
        end_time = datetime.datetime.now()
        
        avg_clustering_score = sum([board["clustering_score"] for board in new_boards])/len(new_boards)
        avg_interaction = sum([board.get("interaction_complexity", 0) for board in new_boards])/len(new_boards)
        avg_empty_cells = sum([board.get("empty", 0) for board in new_boards])/len(new_boards)
        avg_total_score = sum([board.get("total_score", 0) for board in new_boards])/len(new_boards)
        
        print(f"[{datetime.datetime.now()}] Generated {len(new_boards)} new boards for difficulty {diff}")
        print(f"  Avg total score: {avg_total_score:2f}\t Avg clustering score: {avg_clustering_score}\t Avg interaction: {avg_interaction:.2f}\t Avg empty cells: {avg_empty_cells:2f}")
        print(f"  Generation time: {end_time - start_time} seconds")
        
        for board in new_boards:
            with open(f"../Levels/Level{lvlid}.tscn", "w") as f:
                f.write(board["board"].to_scene(lvlid, False))
            lvlid += 1

[2025-03-08 17:40:37.119592] Starting
[2025-03-08 17:40:42.844216] Generated 5 new boards for difficulty tutorial
  Avg total score: inf	 Avg clustering score: inf	 Avg interaction: 0.00	 Avg empty cells: 0.000000
  Generation time: 0:00:05.724624 seconds
[2025-03-08 17:43:31.887665] Generated 15 new boards for difficulty very_easy
  Avg total score: 471.482294	 Avg clustering score: 249.79340554093477	 Avg interaction: 10.84	 Avg empty cells: 0.000000
  Generation time: 0:02:49.043449 seconds
[2025-03-08 17:45:07.911895] Generated 15 new boards for difficulty very_easy2
  Avg total score: 449.602986	 Avg clustering score: 239.85631975953373	 Avg interaction: 9.87	 Avg empty cells: 1.000000
  Generation time: 0:01:36.020230 seconds
[2025-03-08 17:46:47.671121] Generated 15 new boards for difficulty easy
  Avg total score: 514.764996	 Avg clustering score: 271.7395996005105	 Avg interaction: 24.71	 Avg empty cells: 0.800000
  Generation time: 0:01:39.755224 seconds
[2025-03-08 17:48:02.